In [64]:
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import webbrowser
import time
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from git_ignore.config import *
import plotly.graph_objects as go
import re

In [9]:
auth = tweepy.OAuthHandler(twitter_consumer_key, twitter_consumer_secret, twitter_callback_uri)
redirect_url = auth.get_authorization_url()

In [10]:
# auth.get_access_token(user_pin_input)
api = tweepy.API(auth)


# Get popular/recent tweets from keyword search

In [11]:
tweets = tweepy.Cursor(api.search, q='Drake', result_type='popular').items(10)

keyword_df = pd.DataFrame()
name = []
location = []
text = []

for tweet in tweets:
    name.append(tweet.user.name)
    location.append(tweet.user.location)
    text.append(tweet.text)

keyword_df['name'] = name
keyword_df['location'] = location
keyword_df['text'] = text

keyword_df

,name,location,text
0,Mrs. Petty,Queendom,Thank you so much for fkn w|me on live yall. #...
1,$Oxtail is a hustle,Toronto,New Drake Nicki and Wayne on the same night as...
2,Complex Music,,YMCMB BACK.\n\nNicki announces a new collab wi...
3,concrete boy boat,,Drake so fye lol
4,Sizwe Dhlomo,,"Nicki, Drake &amp; Wayne... It’s feeling like ..."
5,TIDAL,,"For the first time ever, @NICKIMINAJ's classic..."
6,XXL Magazine,,"“Seeing Green” by Nicki Minaj, Drake and Lil W..."
7,Mula,,THE BIG 3 🤑🤑🤑 TRIPLE GOATED 🐐🐐🐐 #BEAMMEUPSCOTT...
8,Mula,,SEEING GREEN OUT NOW 🤑🔥🐐\n\n@NICKIMINAJ ft. @L...
9,†,,"Wayne,Nicki,Drake \n\nthey understood the assi..."


In [12]:


fig = go.Figure()

fig.add_trace(go.Table(
        header=dict(values=list(keyword_df.columns),
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[keyword_df.name, keyword_df.location, keyword_df.text],
                   fill_color='lavender',
                   align='left'))
    )


fig.show()

# Get tweets from keyword search

# Get location heatmap using location woeid

# Trending tweets from location keyword

In [13]:
cwoeid = 23424775
woeid = 1
  
# fetching the trends
rj_trends = api.trends_place(id = cwoeid)
trends = []
for trend in rj_trends[0]['trends']: 
    if trend['tweet_volume'] is not None and trend['tweet_volume'] > 10000: 
        trends.append((trend['name'], trend['tweet_volume']))

trends.sort(key=lambda x:-x[1])
trends
# rj_trends

[('The CDC', 598584),
 ('J Cole', 483028),
 ('Congress', 317179),
 ('LEAN ON US JUNGKOOK', 209476),
 ('olivia rodrigo', 190225),
 ('Marjorie Taylor Greene', 137821),
 ('Park Jimin', 137754),
 ('pithani helped murder sushant', 78423),
 ('Tank', 61019),
 ('Phil', 41842),
 ('Elise Stefanik', 40555),
 ('Mikasa', 38344),
 ('Toronto', 35090),
 ('Geriatric', 15616),
 ('Gen X', 15085),
 ('George Lucas', 12611),
 ('#askASHE', 11503)]

In [14]:
import plotly.express as px
df = px.data.gapminder()
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,ZWE,716
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,ZWE,716
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,ZWE,716
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,ZWE,716


## Personal Tweets and Analysis

In [10]:
timeline = api.home_timeline()
for tweet in timeline:
    print(f"{tweet.user.name} said {tweet.text}")

Imran Khan said انسانیت کی تمام حدود اور عالمی قوانین پامال کرتے ہوئے خصوصاً رمضان المبارک کے دوران اسرائیلی فوجیوں کے قبلہ اول یعن… https://t.co/1KzxeLmzI2
Imran Khan said Strongly condemn Israeli Forces' attack esp during Ramazan on Palestinians in Qibla-e-Awaal, Al-Aqsa Mosque, violat… https://t.co/UpMZWC1iuP
SpaceX said Deployment of 60 Starlink satellites confirmed https://t.co/RarbrcfUml
Narendra Modi said Saddened by the demise of INA veteran Lalti Ram Ji. His courage and contributions to India’s freedom struggle will… https://t.co/E5hKmAdkXW
SpaceX said Falcon 9’s first stage has landed on the Just Read the Instructions droneship, completing this booster’s 10th launc… https://t.co/48pJZyHq1i
SpaceX said Liftoff! https://t.co/MZ4ct2Spnj
SpaceX said Watch the first tenth flight of an orbital class rocket booster → https://t.co/bJFjLCzWdK  https://t.co/MDM6RNlN4J
enzo said RT @Abz4ktt: I beg (if it is Laylatul Qadr tonight Inshallah) all of us Muslimeen in the world rn to all rais

# Create an NLP from twitter API

In [108]:
def percentage(part,whole):
 return 100 * float(part)/float(whole)
 
tweet_limit = 50
keyword = 'india'
tweets = tweepy.Cursor(api.search, q=keyword, result_type='popular').items(tweet_limit)
tweet_list = []
negative_list = []
neutral_list = []
positive_list = []
polarity = 0
positive = 0
negative = 0
neutral = 0
for tweet in tweets:
    tweet_list.append(tweet.text)
    analysis = TextBlob(tweet.text)
    score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    compound = score['compound']
    polarity += analysis.sentiment.polarity
    m = max(neg, neu, pos)
    if m == neg:
        negative_list.append(tweet.text)
        negative += 1
    elif m == neu:
        neutral_list.append(tweet.text)
        neutral += 1
    else:
        positive_list.append(tweet.text)
        positive += 1
positive = percentage(positive, tweet_limit)
negative = percentage(negative, tweet_limit)
neutral = percentage(neutral, tweet_limit)
polarity = percentage(polarity, tweet_limit)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

In [109]:
tweet_list = pd.DataFrame(tweet_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
print('total number: ',len(tweet_list))
print('positive number: ',len(positive_list))
print("negative number: ", len(negative_list))
print('neutral number: ',len(neutral_list))

total number:  50
positive number:  0
negative number:  0
neutral number:  50


In [87]:
tweet_list
# tweet_list.drop_duplicates(inplace = True)

,0
0,⚠️ King Vajiralongkorn was rushed to intensive...
1,Breaking. NBC News confirms: The CDC will anno...
2,As repórteres Raquel Krähenbühl e Carolina Cim...
3,- O Brasil é um dos países que mais distribuem...
4,मैं कभी कभी सोचता हूँ कि वो कौन अफ़सर होता होग...
5,18 milhões de doses de vacina contra Covid rec...
6,A lot of COVID deaths in India in young people...
7,Let's review (all CDC-based numbers):\n\n• 99....
8,Today is a great day for America in our long b...
9,The Home Office needs to ask itself hard quest...


In [75]:
tweet_list

,0
0,Thank you so much for fkn w|me on live yall. #...
1,New Drake Nicki and Wayne on the same night as...
2,YMCMB BACK.\n\nNicki announces a new collab wi...
3,Drake so fye lol
4,"Nicki, Drake &amp; Wayne... It’s feeling like ..."


In [68]:
# #Creating new dataframe and new features
# tw_list = pd.DataFrame(tweet_list)
# tw_list["text"] = tw_list[0]
# #Removing RT, Punctuation etc
# remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
# rt = lambda x: re.sub("(@[A-Za-z0–9]+)|([⁰-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
# tw_list["text"] = tw_list.text.map(remove_rt).map(rt)
# tw_list["text"] = tw_list.text.str.lower()
# tw_list.head(10)

error: bad character range ⁰-9 at position 18